# Lecture 4 - Nested CV for CART model

## Load the CCHS data form epi7913A package

In [1]:
library(magrittr)

full_data<-epi7913A::cchs %>% dplyr::slice_sample(prop=0.1)
head(full_data)

age sex CANHEARTbin householdsize education maritalstatus immigration
1 1   2   1           4             3         1             0          
2 1   1   1           5             1         1             0          
3 6   2   1           2             4         3             1          
4 7   2   0           1             2         2             0          
5 5   1   1           2             4         1             0          
6 6   1   0           2             1         3             1          
  houseincome
1 2          
2 1          
3 2          
4 3          
5 2          
6 2

## Define the outcome variable

In [2]:
# define the outcome variable
voutcome <- "CANHEARTbin"

## Iterate using 5-fold cross validation (outer loop) calling the 5-fold CV to tune the model each time (inner loop)

In [3]:
ll.mean<-mean(sapply(caret::createFolds(full_data[, voutcome], k=5), function(x)
{
  testInds <- x
  trnInds <- setdiff(1:nrow(full_data), testInds)
  train_data <- full_data[trnInds,] 
  test_data <- full_data[testInds,]
  
  best_model<-sdgm::cart.bestmodel.bin(train_data, voutcome, n_iter=5)

  preds<-predict(best_model, test_data)
  
  if (!is.null(preds))
  {
    test_ll<- MLmetrics::LogLoss(preds, test_data[,voutcome] ) 
  } else  {
    test_ll<-NA
    print("Logloss calculation failed")
  }
}))
print(ll.mean, na.rm=T)


 Best Parameters Found: 
Round = 5	minsplit = 14.0000	minbucket = 14.0000	cp = 0.01933486	maxdepth = 6.0000	Value = -0.6265288 

 Best Parameters Found: 
Round = 2	minsplit = 15.0000	minbucket = 19.0000	cp = 0.01582232	maxdepth = 9.0000	Value = -0.6289619 

 Best Parameters Found: 
Round = 21	minsplit = 6.0000	minbucket = 16.0000	cp = 0.009291822	maxdepth = 3.0000	Value = -0.6332285 

 Best Parameters Found: 
Round = 7	minsplit = 18.0000	minbucket = 14.0000	cp = 0.00881129	maxdepth = 5.0000	Value = -0.6198323 

 Best Parameters Found: 
Round = 25	minsplit = 16.0000	minbucket = 16.0000	cp = 0.02006107	maxdepth = 14.0000	Value = -0.6275589 
[1] 0.6405565


## Now train the final model

In [4]:
  final_model<-sdgm::cart.bestmodel.bin(full_data, voutcome, n_iter=5)


 Best Parameters Found: 
Round = 3	minsplit = 16.0000	minbucket = 16.0000	cp = 0.01660139	maxdepth = 8.0000	Value = -0.6198392 
